In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np



#this functions will call and return targeted data
"""this function will look for specific tags and there id or class on page and find specific data
    this written in try and except as data might differ for every page then error raise and process might
    intrupted so to loop work in countine it is used"""
def get_title(soup):
    
    try:
        title_raw = soup.find('span', attrs={'id':'productTitle'})
        title_string = title_raw.text
        title = title_string.strip()
    except AttributeError:
        title = ""
    return title
def get_price(soup):
    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()
    except AttributeError:
        price = ""
    return price
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
    return available

In [2]:
#setting user-agent 
HEADER = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36','Accept-Language':'em-US,en,q=0.5'})

link_list =[]
for i in range(1,21):
    #url of targeted page 
    url = f"https://www.amazon.in/s?k=iphone&page={i}"
    #http request 
    webpage = requests.get(url, headers=HEADER)
    #html of targeted page facted
    soup = BeautifulSoup(webpage.content,"html.parser")
    
    links = soup.find_all('a', attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    for link in links:
        link_list.append(link.get('href'))
#empty dic
d = {'title':[],'price':[],'reivew':[],'availability':[]}
#looping through collected link and opening them one by one 
for link in link_list:
    sub_page = requests.get('https://www.amazon.in'+link , headers=HEADER)
    new_soup = BeautifulSoup(sub_page.content,'html.parser')
    #appending data into dic 
    d['title'].append(get_title(new_soup))
    d['price'].append(get_price(new_soup))
    d['reivew'].append(get_rating(new_soup))
    d['availability'].append(get_availability(new_soup))

#converting dic into dataframe and then into csv file 
amazon_df = pd.DataFrame.from_dict(d)
amazon_df.to_csv('amazon_data.csv',header=True,index=False)

In [3]:
amazon_df.shape

(322, 4)

In [4]:
amazon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         322 non-null    object
 1   price         322 non-null    object
 2   reivew        322 non-null    object
 3   availability  322 non-null    object
dtypes: object(4)
memory usage: 10.2+ KB


In [5]:
amazon_df.describe()

,title,price,reivew,availability
count,322,322,322,322
unique,249,88,27,11
top,"iQOO Neo 6 5G (Dark Nova, 8GB RAM, 128GB Stora...","₹2,499.00",Previous page of related Sponsored Products,Only 2 left in stock.
freq,6,29,81,110


In [6]:
amazon_df

,title,price,reivew,availability
0,Apple iPhone 11 (64GB) - Black,Page 1 of 1,Previous page,Currently unavailable.
1,Apple iPhone 13 Mini (256GB) - (Product) RED,"₹65,900.00",4.5 out of 5 stars,In stock.
2,Apple iPhone 14 Pro 256GB Space Black,"₹1,39,900.00",4.6 out of 5 stars,
3,Apple iPhone 12 (64GB) - (Product) RED,"₹49,900.00",4.5 out of 5 stars,Only 1 left in stock.
4,Apple iPhone 14 Plus 256GB Purple,"₹97,530.00",4.7 out of 5 stars,In stock.
...,...,...,...,...
317,"Nillkin Case for Apple iPhone 13 (6.1"" Inch) C...","₹3,499.00",Previous page,Only 2 left in stock.
318,Nillkin Case for Apple iPhone 11 Pro Super Fro...,₹999.00,Previous page of related Sponsored Products,Only 1 left in stock.
319,"Nillkin Case for Apple iPhone 14 Plus (6.7"" In...","₹1,396.00",Previous page,In stock.
320,"Nillkin Case for Apple iPhone 14 Pro Max (6.7""...","₹4,199.00",Previous page of related Sponsored Products,Only 1 left in stock.
